## model

### BILSTM

In [6]:
import keras
import pickle
import numpy as np
import pandas as pd
from tflearn.data_utils import pad_sequences
from collections import defaultdict
from sklearn.metrics import f1_score, precision_score, recall_score
from keras.layers import *
from keras.models import Sequential
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from keras import initializers
import os
import keras
import jieba
from collections import Counter 
jieba.add_word('花呗')
jieba.add_word('借呗')
jieba.add_word('收钱码')
jieba.add_word('收款码')

from keras.optimizers import Adadelta
import gc

config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)

EMBEDDING_DIM = 300
VOCAB_LENGTH = 3000

from tensorflow.python.client import device_lib


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.250 seconds.
Prefix dict has been built succesfully.


In [11]:
train_data_word = pd.read_csv('../train-w_split.csv', header=None)
train_data_char = pd.read_csv('../train-c_split.csv', header=None)
print train_data_word.head()

   0                                     1                                                  2  3
0  1                        11 239 3 2 216  6 5 3 2 19 151 5 216 9 11 965 582 44 5 31 5 17...  1
1  2  105 153 34 3 2 9 72 586 8 139 3241 8                             829 135 139 211 3 2 35  0
2  3               3 2 113 182 12 23 21 10                           6 5 202 14 21 3 2 123 10  0
3  4                             57 54 4 2                                      67 568 54 4 2  0
4  5                           3 2 313 579                                212 1037 14 33 2 10  0


In [10]:
jieba.lcut('花呗分期查询'.decode('UTF-8'))

[u'\u82b1\u5457', u'\u5206\u671f', u'\u67e5\u8be2']

In [10]:

PAD_ID = 0
UNK_ID=1

def translate(text, translation):
    for token, replacement in translation.items():
        text = text.replace(token, ' ' + replacement + ' ')
    text = text.replace('  ', ' ')
    return text
def token_string_as_list(string, token_string='char'):
    string = string.decode('UTF-8')
    translation = {
        '***':'*',
        u'花被':u'花呗'
    }
    translate(string, translation)
    length = len(string)
    if token_string == 'char':
        listt = [string[i] for i in range(length)]
    elif token_string == 'word':
        listt = jieba.lcut(string)
    elif token_string == 'pinyin':
        string = ''.join(jibe.lcut(string))
        listt = ''.join(lazy_pinyin(string)).split()
    listt = [item for item in listt if item.strip()]
    return listt
def create_voca(train_data, token_string='char', voca_size=5000):
    size = train_data.shape[0]
    input_count = Counter()
    for i in range(size):
        token_list = token_string_as_list(train_data.iloc[i, 1], token_string=token_string)
        input_count.update(token_list)
        token_list = token_string_as_list(train_data.iloc[i, 2], token_string=token_string)
        input_count.update(token_list)
    vocab_worddict = {}
    vocab_indexdict = {}
    vocab_list = input_count.most_common(voca_size)
    for i, tuplee in enumerate(vocab_list):
        word,_ = tuplee
        vocab_worddict[i+2] = word
        vocab_indexdict[word] = i+2
    return vocab_worddict, vocab_indexdict
def parse_train_data(train_data, vocab_indexdict, token_string):
    size = train_data.shape[0]
    input_count = Counter()
    parse_data = train_data.copy()
    temp_data = train_data.copy()
    for i in range(size):
        token_list1 = token_string_as_list(train_data.iloc[i, 1], token_string=token_string)
        token_list2 = token_string_as_list(train_data.iloc[i, 2], token_string=token_string)
        temp_data.iloc[i, 1] = ' '.join(token_list1)
        temp_data.iloc[i, 2] = ' '.join(token_list2)
        #print(token_list1, train_data.iloc[i, 1].decode('UTF-8'))
        parse_data.iloc[i, 1] = ' '.join([str(vocab_indexdict[item]) for item in token_list1 if item in vocab_indexdict])
        parse_data.iloc[i, 2] = ' '.join([str(vocab_indexdict[item]) for item in token_list2 if item in vocab_indexdict])
    return parse_data



def parse_split(train_data):
    vocab_worddict_word, vocab_indexdict_word = create_voca(train_data, token_string='char')
    parse_data = parse_train_data(train_data, vocab_indexdict_word,token_string='char')
    return parse_data

train_data_char_df = parse_split(train_data_char)

In [11]:
def parse_split2(train_data):
    vocab_worddict_word, vocab_indexdict_word = create_voca(train_data, token_string='word')
    parse_data = parse_train_data(train_data, vocab_indexdict_word,token_string='word')
    return parse_data
train_data_word_df = parse_split2(train_data_word)

In [12]:
train_data_word_df.head()

,0,1,2,3
0,1,11 239 3 2 216,6 5 3 2 19 151 5 216 9 11 965 582 44 5 31 5 17...,1
1,2,105 153 34 3 2 9 72 586 8 139 3241 8,829 135 139 211 3 2 35,0
2,3,3 2 113 182 12 23 21 10,6 5 202 14 21 3 2 123 10,0
3,4,57 54 4 2,67 568 54 4 2,0
4,5,3 2 313 579,212 1037 14 33 2 10,0


In [13]:
train_data_char_df.to_csv('../train-c_split.csv', index=False, header=False)
train_data_word_df.to_csv('../train-w_split.csv', index=False, header=False)

In [29]:

VOC_SIZE_CHAR = 2087
VOC_SIZE_WORD = 5002
from keras.layers import *
from keras.activations import softmax
from keras.models import Model

def unchanged_shape(input_shape):
    return input_shape

def time_distributed(x, layers):
    for l in layers:
        x = TimeDistributed(l)(x)
    return x

def align(input_1, input_2):
    attention = Dot(axes=-1)([input_1, input_2])
    w_att_1 = Lambda(lambda x: softmax(x, axis=1),
                     output_shape=unchanged_shape)(attention)
    w_att_2 = Permute((2,1))(Lambda(lambda x: softmax(x, axis=2),
                             output_shape=unchanged_shape)(attention))
    in1_aligned = Dot(axes=1)([w_att_1, input_1])
    in2_aligned = Dot(axes=1)([w_att_2, input_2])
    return in1_aligned, in2_aligned

def aggregate(x1, x2, x10, x20,  num_class, dense_dim=300, dropout_rate=0.2, activation="relu"):#
    feat1 = concatenate(list(map(lambda l: l(x1), [GlobalAvgPool1D(), GlobalMaxPool1D()])))
    feat2 = concatenate(list(map(lambda l: l(x2), [GlobalAvgPool1D(), GlobalMaxPool1D()])))
    feat10= concatenate(list(map(lambda l: l(x10), [GlobalAvgPool1D(), GlobalMaxPool1D()])))
    feat20 = concatenate(list(map(lambda l: l(x20), [GlobalAvgPool1D(), GlobalMaxPool1D()])))
    x = Concatenate()([feat1, feat2, feat10, feat20]) #
    x = BatchNormalization()(x)
    x = Dense(dense_dim, activation=activation)(x)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)
    x = Dense(dense_dim, activation=activation)(x)
    x = Dropout(dropout_rate)(x)
    return x  

def abs_diff(X):
    s = X[0]
    for i in range(1, len(X)):
        s -= X[i]
    s = K.abs(s)
    return s

def aggregate_both(x1, x2, num_class, dense_dim=300, dropout_rate=0.2, activation="relu"):#
    feat1 = concatenate(list(map(lambda l: l(x1), [GlobalAvgPool1D(), GlobalMaxPool1D()])))
    feat2 = concatenate(list(map(lambda l: l(x2), [GlobalAvgPool1D(), GlobalMaxPool1D()])))
    feat3 = add([feat1, feat2])
    feat4 = subtract([feat1, feat2])
    feat5 = Lambda(lambda x: abs(x[0]-x[1]))([feat1, feat2])
    x = Concatenate()([feat1, feat2]) #, feat3, feat4
    x = BatchNormalization()(x)
    x = Dense(dense_dim, activation=activation)(x)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)
    x = Dense(dense_dim, activation=activation)(x)
    x = Dropout(dropout_rate)(x)
    return x

def embed_project_atten(q1, q2, encode, projection_dim=300, projection_hidden=200, projection_dropout=0.2,
                           compare_dim=500, compare_dropout=0.2,
                           dense_dim=300, dropout_rate=0.2,
                           lr=1e-3, activation='relu'):
    q1_embed = encode(q1)
    q2_embed = encode(q2)
    
    # Projection
    projection_layers = []
    if projection_hidden > 0:
        projection_layers.extend([
                Dense(projection_hidden, activation=activation),
                Dropout(rate=projection_dropout),
            ])
    projection_layers.extend([
            Dense(projection_dim, activation=None),
            Dropout(rate=projection_dropout),
        ])
    q1_encoded = time_distributed(q1_embed, projection_layers)
    q2_encoded = time_distributed(q2_embed, projection_layers)
    
    # Attention
    q1_aligned, q2_aligned = align(q1_encoded, q2_encoded)    
    
    # Compare
    q1_combined = concatenate([q1_encoded, q2_aligned])
    q2_combined = concatenate([q2_encoded, q1_aligned]) 
    compare_layers = [
        Dense(compare_dim, activation=activation),
        Dropout(compare_dropout),
        Dense(compare_dim, activation=activation),
        Dropout(compare_dropout),
    ]
    q1_compare = time_distributed(q1_combined, compare_layers)
    q2_compare = time_distributed(q2_combined, compare_layers)
    return q1_compare, q2_compare

def lstm_project_atten(q1_embed, q2_embed, projection_dim=300, projection_hidden=200, projection_dropout=0.2,
                           compare_dim=500, compare_dropout=0.2,
                           dense_dim=300, dropout_rate=0.2,
                           lr=1e-3, activation='relu'):
    
    # CuDNN
    lstm0 = CuDNNLSTM(model_params['num_lstm'],
                 return_sequences = True)
    lstm1 = Bidirectional(CuDNNLSTM(model_params['num_lstm'],
                 return_sequences = True))
    lstm2 = CuDNNLSTM(model_params['num_lstm'])
    
    att1 = Lambda(lambda x: K.max(x,axis = 1))
    q1_embed = lstm1(q1_embed)
    q2_embed = lstm1(q2_embed)
    
    # Projection
    projection_layers = []
    if projection_hidden > 0:
        projection_layers.extend([
                Dense(projection_hidden, activation=activation),
                Dropout(rate=projection_dropout),
            ])
    projection_layers.extend([
            Dense(projection_dim, activation=None),
            Dropout(rate=projection_dropout),
        ])
    q1_encoded = time_distributed(q1_embed, projection_layers)
    q2_encoded = time_distributed(q2_embed, projection_layers)
    
    # Attention
    q1_aligned, q2_aligned = align(q1_encoded, q2_encoded)  
    
    # Compare
    q1_combined = concatenate([q1_encoded, q2_aligned])
    q2_combined = concatenate([q2_encoded, q1_aligned]) 
    compare_layers = [
        Dense(compare_dim, activation=activation),
        Dropout(compare_dropout),
        Dense(compare_dim, activation=activation),
        Dropout(compare_dropout),
    ]
    q1_compare = time_distributed(q1_combined, compare_layers)
    q2_compare = time_distributed(q2_combined, compare_layers)
    return q1_compare, q2_compare

def build_model_combine(params, num_class=1,projection_dim=300, projection_hidden=200, projection_dropout=0.2,
                           compare_dim=500, compare_dropout=0.2,
                           dense_dim=300, dropout_rate=0.2,
                           lr=1e-3, activation='relu'):
    q1 = Input(name='q1',shape=(MAX_SEQUENCE_LENGTH_CHAR,), dtype='int32')
    q2 = Input(name='q2',shape=(MAX_SEQUENCE_LENGTH_CHAR,), dtype='int32')
    
    q1_com = Input(name='q1_com',shape=(MAX_SEQUENCE_LENGTH_CHAR, 1), dtype='float32')
    q2_com = Input(name='q2_com',shape=(MAX_SEQUENCE_LENGTH_CHAR, 1), dtype='float32')
    
    encode = Embedding(VOC_SIZE_CHAR+2, 300, input_length = MAX_SEQUENCE_LENGTH_CHAR) # 
    #encode0 = StaticEmbedding(embedding_matrix_char) #
    
    # Word Representation Layer
    q1_embed = encode(q1) #concatenate([, q1_com])
    q2_embed = encode(q2) #concatenate([, q2_com])
    
    # Densely-connected Co-attentive networks
    q1_compare0, q2_compare0 = lstm_project_atten(q1_embed, q2_embed)
    
    
    q1_compare0 = concatenate([q1_compare0, q1_embed])
    q2_compare0 = concatenate([q2_compare0, q2_embed])
    
    q1_compare1, q2_compare1 = lstm_project_atten(q1_compare0, q2_compare0)
    
    q1_compare1 = concatenate([q1_compare1, q1_compare0])
    q2_compare1 = concatenate([q2_compare1, q2_compare0])
    
    q1_compare2, q2_compare2 = lstm_project_atten(q1_compare1, q2_compare1)
    
    
    # Aggregate
    x = aggregate_both(q1_compare1, q2_compare1, num_class)#, q10_compare1, q20_compare1
    #x1 = aggregate(q1_embed, q2_embed, q10_embed, q20_embed, num_class)#
    #x = concatenate([x0, x1])
    scores = Dense(num_class, activation='sigmoid')(x)
    
    model = Model(inputs=[q1, q2, q1_com, q2_com], outputs=scores)#
    return model

def build_model(params, num_class=1,projection_dim=300, projection_hidden=200, projection_dropout=0.2,
                           compare_dim=500, compare_dropout=0.2,
                           dense_dim=300, dropout_rate=0.2,
                           lr=1e-3, activation='relu'):
    q1 = Input(name='q1',shape=(MAX_SEQUENCE_LENGTH_CHAR,), dtype='int32')
    q2 = Input(name='q2',shape=(MAX_SEQUENCE_LENGTH_CHAR,), dtype='int32')
    q10 = Input(name='q10',shape=(MAX_SEQUENCE_LENGTH_WORD, ), dtype='int32')
    q20 = Input(name='q20',shape=(MAX_SEQUENCE_LENGTH_WORD, ), dtype='int32')
    
    #  #Embedding Embedding(VOC_SIZE_CHAR, 300, input_length = MAX_SEQUENCE_LENGTH_CHAR)  Embedding(VOC_SIZE_WORD, 300, input_length = MAX_SEQUENCE_LENGTH_WORD) StaticEmbedding(embedding_matrix_char)
    encode = Embedding(VOC_SIZE_CHAR+2, 300, input_length = MAX_SEQUENCE_LENGTH_CHAR) # StaticEmbedding(embedding_matrix_char) #
    q1_compare, q2_compare = embed_project_atten(q1, q2, encode)
    
    
    encode = Embedding(VOC_SIZE_WORD+2, 300, input_length = MAX_SEQUENCE_LENGTH_WORD)  # StaticEmbedding(embedding_matrix_word) # 
    q10_compare, q20_compare = embed_project_atten(q10, q20, encode)
    
    
    # Aggregate
    scores = aggregate(q1_compare, q2_compare, q10_compare, q20_compare, num_class)#
    
    scores = Dense(num_class, activation='sigmoid')(scores)
    model = Model(inputs=[q1, q2, q10, q20], outputs=scores)#
    return model

In [14]:
train_data_char_df = train_data_char

In [15]:
train_data_char_df.head()

,0,1,2,3
0,1,1517 15 4 186 128 3 2 56 72 43 59,7 9 3 2 18 23 52 9 56 72 43 59 14 15 4 186 128...,1
1,2,159 31 13 10 3 2 14 94 65 88 10 199 123 195 10,563 9 178 199 94 18 3 2 25 8,0
2,3,3 2 154 131 23 51 5 21 31 36 16,7 9 242 212 22 23 31 36 3 2 6 8 16,0
3,4,76 84 259 226 68 95 6 2,91 459 141 68 95 6 2,0
4,5,3 2 230 59 25 34,179 270 59 230 680 22 23 11 3 2 16,0


In [30]:
word_squence_ques1_char = list(train_data_char_df.iloc[:, 1])
word_squence_ques1_char = [[int(im) for im in item.split(' ')] for item in word_squence_ques1_char]
word_squence_ques2_char = list(train_data_char_df.iloc[:, 2])
word_squence_ques2_char = [[int(im) for im in item.split(' ')] for item in word_squence_ques2_char]

# word_squence_ques1_word = list(train_data_word.iloc[:, 1])
# word_squence_ques1_word = [[int(im) for im in item.split(' ')] for item in word_squence_ques1_word]
# word_squence_ques2_word = list(train_data_word.iloc[:, 2])
# word_squence_ques2_word = [[int(im) for im in item.split(' ')] for item in word_squence_ques2_word]


MAX_SEQUENCE_LENGTH_WORD = 30 # char 40 word 30
MAX_SEQUENCE_LENGTH_CHAR = 40 # char 40 word 30
word_squence_ques1_char = pad_sequences(word_squence_ques1_char, maxlen=MAX_SEQUENCE_LENGTH_CHAR)
word_squence_ques2_char = pad_sequences(word_squence_ques2_char, maxlen=MAX_SEQUENCE_LENGTH_CHAR)
def cal_comman_sen(word_squence_ques1_char, word_squence_ques2_char):
    comman_sen1 = []
    comman_sen2 = []
    for ques1, ques2 in zip(word_squence_ques1_char, word_squence_ques2_char):
        comman_sen1.append([[1] if item in ques2 and item!=0 else [0] for item in ques1])
        comman_sen2.append([[1] if item in ques1 and item!=0 else [0] for item in ques2])
    return comman_sen1, comman_sen2
comman_sen1, comman_sen2 = cal_comman_sen(word_squence_ques1_char, word_squence_ques2_char)
# word_squence_ques1_word = pad_sequences(word_squence_ques1_word, maxlen=MAX_SEQUENCE_LENGTH_WORD)
# word_squence_ques2_word = pad_sequences(word_squence_ques2_word, maxlen=MAX_SEQUENCE_LENGTH_WORD)


In [ ]:
class Metrics(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.validation_data[:-3])))#
        val_label = self.validation_data[-3]
        for weight in range(10, 90, 5):
            val_pred = [1 if item > weight*0.01 else 0 for item in val_predict]
            _val_f1 = f1_score(val_label, val_pred)
            print("epoch end ", weight*0.01, ", f1-score: ", _val_f1, "precision: ", 
                  precision_score(val_label, val_pred, average='binary'), "recall: ", 
                  recall_score(val_label, val_pred, average='binary'))
metrics = Metrics()

from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
X_train_q1 = word_squence_ques1_char
X_train_q2 = word_squence_ques2_char
X_train_q1_com = np.array(comman_sen1)
X_train_q2_com = np.array(comman_sen2)
y_train = train_data_char_df.iloc[:, 3]

class_weight = {0: 1.,
                1: 5.,}

X_fold_val_q1_out,X_fold_val_q2_out, X_fold_val_magic_out = [0, 0, 0]

def model_train(model_func, X_train_q1, X_train_q2, y_train, model_checkpoint_path):
    NUM_FOLDS = 10
    kfold = StratifiedKFold(
        n_splits=NUM_FOLDS,
        shuffle=True,
        random_state=30
    )
    BATCH_SIZE = 256
    MAX_EPOCHS = 100

    for fold_num, (ix_train, ix_val) in enumerate(kfold.split(X_train_q1, y_train)):

        # Augment the training set by mirroring the pairs.
        X_fold_train_q1 = np.vstack([X_train_q1[ix_train], X_train_q2[ix_train]])
        X_fold_train_q2 = np.vstack([X_train_q2[ix_train], X_train_q1[ix_train]])
        X_fold_train_q1_com = np.vstack([X_train_q1_com[ix_train], X_train_q2_com[ix_train]])
        X_fold_train_q2_com = np.vstack([X_train_q2_com[ix_train], X_train_q1_com[ix_train]])
#         X_fold_train_q10 = np.vstack([X_train_q10[ix_train], X_train_q20[ix_train]])
#         X_fold_train_q20 = np.vstack([X_train_q20[ix_train], X_train_q10[ix_train]])
        #X_fold_train_magic = np.vstack([X_train_magic[ix_train], X_train_magic[ix_train]])

        X_fold_val_q1 = np.vstack([X_train_q1[ix_val], X_train_q2[ix_val]])
        X_fold_val_q2 = np.vstack([X_train_q2[ix_val], X_train_q1[ix_val]])
        X_fold_val_q1_com = np.vstack([X_train_q1_com[ix_val], X_train_q2_com[ix_val]])
        X_fold_val_q2_com = np.vstack([X_train_q2_com[ix_val], X_train_q1_com[ix_val]])
#         X_fold_val_q10 = np.vstack([X_train_q10[ix_val], X_train_q20[ix_val]])
#         X_fold_val_q20 = np.vstack([X_train_q20[ix_val], X_train_q10[ix_val]])
        #X_fold_val_magic = np.vstack([X_train_magic[ix_val], X_train_magic[ix_val]])
        
        
        # Ground truth should also be "mirrored".
        y_fold_train = np.concatenate([y_train[ix_train], y_train[ix_train]])
        y_fold_val = np.concatenate([y_train[ix_val], y_train[ix_val]])

        print('Fitting fold')

        # Compile a new model.
        # model1 = create_model_cnn(model_params)
        model = model_func(model_params)

        model.compile(optimizer= keras.optimizers.Adam(lr = 0.001), loss="binary_crossentropy", 
            metrics=['accuracy'])
        # Train.
        model.fit(
            [X_fold_train_q1, X_fold_train_q2, 
             X_fold_train_q1_com, X_fold_train_q2_com,
            # X_fold_train_q10, X_fold_train_q20, 
#              X_fold_train_char0_len, X_fold_train_char1_len,
#              X_fold_train_word0_len, X_fold_train_word1_len,
             # X_fold_train_magic
            ], y_fold_train,#
            validation_data=([
                X_fold_val_q1, X_fold_val_q2, 
             X_fold_val_q1_com, X_fold_val_q2_com,
                           #   X_fold_val_q10, X_fold_val_q20, 
#                               X_fold_val_char0_len, X_fold_val_char1_len,
#                               X_fold_val_word0_len,X_fold_val_word1_len,
                              # X_fold_val_magic
                             ], y_fold_val),#

            batch_size=BATCH_SIZE,
            epochs=MAX_EPOCHS,
            verbose=1,
            shuffle=True,
            class_weight=class_weight,

            callbacks=[
                # Stop training when the validation loss stops improving.
                EarlyStopping(
                    monitor='val_loss',
                    min_delta=0.001,
                    patience=20,
                    verbose=1,
                    mode='auto',
                ),
                ReduceLROnPlateau(
                    monitor='val_loss', 
                    factor=0.1, 
                    patience=5, 
                    min_delta=0.0001,
                    cooldown=1, 
                ),
                # Save the weights of the best epoch.
                ModelCheckpoint(
                    os.path.join(model_checkpoint_path + str(fold_num) + 'weights.{epoch:02d}.hdf5'),
                    monitor='val_loss',
                    save_best_only=False,
                    verbose=2,
                    mode='auto',
                ),
                metrics
            ],
        )
        break
    
    # Clear GPU memory.
    K.clear_session()
    del X_fold_train_q1, X_fold_train_q2, X_fold_train_magic
    del X_fold_val_q1, X_fold_val_q2, X_fold_val_magic
    del model
    gc.collect()
    # return score

model_path = '../data/checkpoints/test/'
# model_train(create_model, X_train_q1, X_train_q2, X_train_magic, y_train, model_path)


model_params = {
    'dense_dropout_rate': 0.3,
    'num_dense': 150,
    'num_lstm': 128,
    'num_filters':32
}

model_train(build_model_combine, X_train_q1, X_train_q2, y_train, model_path)


Fitting fold
Train on 184456 samples, validate on 20498 samples
Epoch 1/100
  5632/184456 [..............................] - ETA: 3:46 - loss: 1.3841 - acc: 0.4361

In [16]:
y_train.value_counts()

0    83792
1    18684
Name: 1.1, dtype: int64

In [17]:
train_data_char_df.shape

(102476, 4)

In [29]:
MAX_SEQUENCE_LENGTH_CHAR

60

In [30]:
MAX_SEQUENCE_LENGTH_WORD

50

In [48]:
list(train_data_char_df.iloc[0, :])

[2,
 '4 22 21 2 29 8 4 16 24 4 17 25 5 11 12 2 6 7 50 29 32 2 45 12 5 19 57 3 62 51 4 17 25 50 29 21 2 46 32 4 17 60 4 17 25',
 '10 20 21 10 13 38 2 14 68 50 29 21 2 45 12 3 14 23 5 11 12 2 6 7 4 18 14 3 35 30',
 0]

In [46]:
train_data_char.head()

,1,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1.1
0,2,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0
1,3,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0
2,4,如何得知关闭借呗,想永久关闭借呗,0
3,5,花呗扫码付钱,二维码扫描可以用花呗吗,0
4,6,花呗逾期后不能分期吗,我这个 逾期后还完了 最低还款 后 能分期吗,0
